In [7]:
import jax
jax.config.update("jax_default_device", jax.devices("cpu")[0])
import foundry.core as F
from foundry.core import tree

from policy_eval.common import Result
from policy_eval.methods.diffusion_policy import Checkpoint
checkpoint : Checkpoint = Result.load_s3("s3://wandb-data/uvn050jd/final_result.zarr")

In [8]:
denoiser = checkpoint.create_denoiser()
env, splits = checkpoint.data.load({"train"})
train_data = splits["train"].as_pytree()

In [9]:
import foundry
import foundry.random

In [ ]:
F.jit
def find_datapoint(rng_key):
    def find_candidate(rng_key):
        i, j = foundry.random.randint(rng_key, (2,), minval=0, maxval=tree.axis_size(train_data, 0))
        return test, i, j


def test_linear(rng_key):
    N = tree.axis_size(train_data, 0)